<a href="https://colab.research.google.com/github/AtulAravindDas/DeepLearningAI_CrewAI_Agents/blob/main/Customer_Outreach_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent,Task,Crew

In [5]:
import os
import getpass

api_key=os.environ.get("OPENAI_API_KEY")

if api_key:
  openai_api_key=api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key already set")
else:
  api_key=getpass.getpass("OpenAI API Key: ")
  openai_api_key=api_key
  serper_api_key=getpass.getpass("SerperDev API Key: ")
  os.environ["SERPER_API_KEY"]=serper_api_key
  os.environ["OPENAI_API_KEY"]=openai_api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key set")
  print("SerperDev API Key Set")

OpenAI API Key: ··········
SerperDev API Key: ··········
OpenAI API Key set
SerperDev API Key Set


# **Creating Agents**

1. Sales Rep agent: Identify high value leads that match ideal customer profiles(ICP)
2. Lead Sales Rep Agent: Find the actual personalized content and build personalized communication

In [6]:
sales_rep_agent=Agent(role="Sales Representative",
                      goal="Identify high-value leads that match our ideal customer profile",
                      backstory=("As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."),allow_delegation=False,verbose=True)

In [7]:
lead_sales_rep_agent=Agent(role="Lead Sales Representative",
                           goal="Nurture leads with personalized, compelling communications",
                           backstory=("Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."),allow_delegation=False,verbose=True)

# **Create Tools**

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **crewai tools**

In [10]:
from crewai_tools import DirectoryReadTool,FileReadTool,SerperDevTool
directory_read_tool = DirectoryReadTool(directory='/content/drive/MyDrive/instructions') #Specifies the local directory of instructions to handle them
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

## **Custom Tools using crewai basetool(a class to be inherited)**

In [11]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
  name: str="Sentiment Analysis Tool"
  description: str=("Analyze the sentiment of a given text"
  "to ensure positive and engaging communication")

  def _run(self,text:str):
    #Run any other type of code in it. THis function runs the necessary functionalities(like call an API, take an action or send an email)
    return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()#Inherits base tool and creates an SentimentAnalysis object

# **Create Tasks**

In [13]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [14]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

# **Creating the crew**

In [15]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

# **Running the crew**

In [16]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To conduct an in-depth analysis of DeepLearningAI, I will first search the internet for relevant information about the company. This will help gather details about their background, key personnel, recent developments, and potential needs. 

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company background key personnel recent developments"} 


Search results: Title: DeepLearnin

In [17]:
from IPython.display import Markdown
Markdown(result)

**Email Draft 1: Introduction and Congratulations**

Subject: Enhancing AI Education with CrewAI's Innovative Solutions

Hi Andrew,

I hope this message finds you well. Firstly, congratulations on the recent success of DeepLearning.AI, particularly your popular courses like "AI For Everyone" and "Generative AI for Everyone." It's inspiring to see how you've reached over 7 million learners globally with your world-class education initiatives.

At CrewAI, we deeply admire your mission to make AI education accessible worldwide. We believe that our solutions could further support your goals by enhancing customer outreach, streamlining educational content delivery, and optimizing learner engagement. Our multi-agent systems can personalize learning experiences, manage AI-driven communication with learners, and analyze learner data to improve course content.

We would love the opportunity to collaborate on AI-driven educational projects or webinars, showcasing how CrewAI's capabilities can enhance DeepLearning.AI's online learning platforms. I would be delighted to arrange a demonstration of our tools, tailored specifically to the needs of education technology companies like yours.

Please let me know a convenient time for us to discuss this further.

Looking forward to the possibility of working together to empower more learners globally.

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI

---

**Email Draft 2: Follow-Up and Value Proposition**

Subject: Empowering AI Education Together

Hi Andrew,

I hope you're having a great week. I wanted to follow up on my previous email regarding a potential collaboration between DeepLearning.AI and CrewAI. We are genuinely excited about the prospect of working together to enhance AI education globally.

At CrewAI, we specialize in leveraging AI-driven solutions to transform educational experiences. Our tools are designed to create more interactive and personalized learning environments, which we believe align perfectly with DeepLearning.AI’s commitment to delivering top-notch AI education.

We envision a partnership where our technology can complement your existing platforms, helping to further engage your learners and improve the overall learning journey. I am keen to demonstrate how our multi-agent systems can specifically benefit DeepLearning.AI and align with your future goals.

Could we schedule a call next week to explore this further? I am confident that together, we can amplify the impact of AI education worldwide.

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI

---

**Email Draft 3: Proposal for Joint Initiatives**

Subject: Exploring Joint Initiatives for AI Talent Development

Hi Andrew,

I trust this message finds you well. As a leader in AI education, DeepLearning.AI has set a remarkable standard in empowering learners across the globe. At CrewAI, we share a similar vision of fostering AI talent and would be thrilled to explore joint initiatives that could benefit both organizations.

We propose collaborating on AI-driven educational projects or webinars that not only showcase CrewAI’s innovative solutions but also highlight DeepLearning.AI’s pioneering efforts in AI education. Together, we can develop programs that increase learner engagement and optimize content delivery through advanced AI technologies.

I would love to hear your thoughts on this proposal and discuss how we can work together to create a more impactful educational experience for AI enthusiasts worldwide. Please let me know your availability for a discussion.

Looking forward to the exciting possibilities that lie ahead.

Warm regards,

[Your Name]  
Lead Sales Representative  
CrewAI

These drafts are designed to initiate a meaningful dialogue with Andrew Ng, aligning CrewAI’s solutions with DeepLearning.AI’s mission and recent achievements.